In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/PixelTransformer/PixelFormer-main/data_splits

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import numpy as np
import os
import shutil
import glob
import argparse
import numpy as np
import random
import plotly
import plotly.figure_factory as ff
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data.distributed
from torchvision import transforms
import time
import numpy as np
from PIL import Image
import os
import random
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#Taken inspiration from - Data processing for nyudepth dataset - referenced  https://arxiv.org/pdf/2210.09071.pdf for preprocessing
class NewDataLoader(torch.utils.data.Dataset):
  def __init__(self, args, mode,transform=None,filename=None):

    self.do_kb_crop = False
    self.input_height = 480
    self.input_width = 640
    self.do_random_rotate = True
    self.degree = 2.5
    self.transform = transform
    self.filenames = None

    filename = filename
    with open(filename, 'r') as f:
        self.filenames = f.readlines()
        
  def __len__(self):
        return len(self.filenames)

  def __getitem__(self,idx):
    if mode =='train':

      sample_path = self.filenames[idx]
      rgb_file = sample_path.split()[0]
      depth_file = sample_path.split()[1]

      image_path = rgb_file
      depth_path = depth_file
      image = Image.open(image_path)
      depth_gt = Image.open(depth_path)
      

      if self.input_height == 480:
        depth_gt = np.array(depth_gt)
        # print("Depth gt",depth_gt)
        valid_mask = np.zeros_like(depth_gt)
        valid_mask[45:472, 43:608] = 1
        depth_gt[valid_mask==0] = 0
        depth_gt = Image.fromarray(depth_gt)
      else:
        depth_gt = depth_gt.crop((43, 45, 608, 472))
        image = image.crop((43, 45, 608, 472))

      if self.do_random_rotate is True:
                random_angle = (random.random() - 0.5) * 2 * self.degree
                image = self.rotate_image(image, random_angle)
                depth_gt = self.rotate_image(depth_gt, random_angle, flag=Image.NEAREST)
                # depth_gt.show()
            
      image = np.asarray(image, dtype=np.float32) / 255.0
      depth_gt = np.asarray(depth_gt, dtype=np.float32)
      depth_gt = np.expand_dims(depth_gt, axis=2)

      depth_gt = depth_gt / 1000.0
      img, depth = image, depth_gt

      #Data processing for nyudepth dataset - referenced  https://arxiv.org/pdf/2210.09071.pdf for preprocessing
      H, W = img.shape[0], img.shape[1]
      a, b, c, d = random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1)
      l, u = int(a*W), int(b*H)
      w, h = int(max((W-a*W)*c*0.75, 1)), int(max((H-b*H)*d*0.75, 1))
      depth_copied = np.repeat(depth, 3, axis=2)
      M = np.ones(img.shape)
      M[l:l+h, u:u+w, :] = 0
      img = M*img + (1-M)*depth_copied
      image = img.astype(np.float32)


      if image.shape[0] != self.input_height or image.shape[1] != self.input_width:
          image, depth_gt = self.random_crop(image, depth_gt, self.input_height, self.input_width)
      image, depth_gt = self.train_preprocess(image, depth_gt)
      sample = {'image': image, 'depth': depth_gt}

      if self.transform:
          sample = self.transform(sample)
      
      return sample

  def rotate_image(self, image, angle, flag=Image.BILINEAR):
        result = image.rotate(angle, resample=flag)
        return result

  def train_preprocess(self, image, depth_gt):
        # Random flipping
        do_flip = random.random()
        if do_flip > 0.5:
            image = (image[:, ::-1, :]).copy()
            depth_gt = (depth_gt[:, ::-1, :]).copy()
    
        # Random gamma, brightness, color augmentation
        do_augment = random.random()
        if do_augment > 0.5:
            image = self.augment_image(image)
    
        return image, depth_gt
  def augment_image(self, image):
        # gamma augmentation
        gamma = random.uniform(0.9, 1.1)
        image_aug = image ** gamma

        brightness = random.uniform(0.75, 1.25)
        image_aug = image_aug * brightness

        colors = np.random.uniform(0.9, 1.1, size=3)
        white = np.ones((image.shape[0], image.shape[1]))
        color_image = np.stack([white * colors[i] for i in range(3)], axis=2)
        image_aug *= color_image
        image_aug = np.clip(image_aug, 0, 1)

        return image_aug


In [ ]:
class ToTensor(object):
    def __init__(self, mode):
        self.mode = mode
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def to_tensor(self, pic):
        if isinstance(pic, np.ndarray):
            img = torch.from_numpy(pic.transpose((2, 0, 1)))
            return img

    def __call__(self, sample):
        image = sample['image']
        image = self.to_tensor(image)
        image = self.normalize(image)
        depth = sample['depth']
        if self.mode == 'train':
            depth = self.to_tensor(depth)
            return {'image': image, 'depth': depth}
        

In [ ]:
from torch.utils.data import Dataset, DataLoader
def preprocessing_transforms(mode):
    return transforms.Compose([
        ToTensor(mode=mode)
    ])
mode = 'train'
batch_size = 512
train_sampler = None
num_threads = 1
train_file = '/content/drive/MyDrive/CS674-Saha/NewCode/train_values.txt'
test_file = '/content/drive/MyDrive/CS674-Saha/NewCode/test_values.txt'
training_samples = NewDataLoader([], mode, transform=preprocessing_transforms(mode),filename=train_file)
testing_samples = NewDataLoader([], mode, transform=preprocessing_transforms(mode),filename=test_file)
train_data = DataLoader(training_samples, batch_size,
                                   shuffle=(train_sampler is None),
                                   num_workers=1,
                                   pin_memory=True,
                                   sampler=train_sampler)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
class silog_loss(nn.Module):
    def __init__(self, variance_focus):
        super(silog_loss, self).__init__()
        self.variance_focus = variance_focus

    def forward(self, depth_est, depth_gt,mask):
        d = torch.log(depth_est[mask]) - torch.log(depth_gt[mask])
        return torch.sqrt((d ** 2).mean() - self.variance_focus * (d.mean() ** 2)) * 10.0


In [ ]:
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta

    def forward(self, y_pred, y_true):
        residual = torch.abs(y_true - y_pred)
        condition = residual < self.delta
        loss = torch.where(condition, 0.5 * residual ** 2, self.delta * residual - 0.5 * self.delta ** 2)
        return torch.mean(loss)

In [ ]:
def berhu_loss(pred, target, threshold=0.2):
    diff = torch.abs(target - pred)
    delta = threshold * torch.max(target).item()
    mask = (diff < delta).float()
    loss = mask * (diff ** 2 / delta) + (1 - mask) * (diff - 0.5 * delta)
    return torch.mean(loss)

Swin Transformer model

In [ ]:
def window_reverse(windows, window_size, H, W):
    #opposite conversion of windows back to the orginial picture
    n = windows.shape[0]
    to_restore_h_w = H * W / window_size / window_size
    N = int( n / to_restore_h_w) #as n=B*num_windows
    H_ =  H // window_size
    W_ = W // window_size
    x = windows.view(N, H_ , W_, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous()
    x = x.view(N, H, W, -1)
    return x
    
def window_partition(x, window_size):
    #convert image to windows and returns number of windows with window_shape and channels
    N, H, W, C = x.shape
    H =  H // window_size
    W = W // window_size
    x = x.view(N, H, window_size, W , window_size, C) #new height and width based on windows
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C) 
    #send windows of shape (num_windows*B, window_size, window_size, C)
    return windows
    

class SwinTransformerBlock(nn.Module):
    def __init__(self, in_channels, window_size=8, shift_size=8):
        super(SwinTransformerBlock,self).__init__()
        self.in_channels = in_channels
        self.window_size = window_size
        self.shift_size = shift_size
        self.num_heads = 8
        self.patch_norm = True
        self.p_size = 4
        self.patch_size = to_2tuple(self.p_size)
        self.embed = 96
        self.dim = 96
        self.height = 480
        self.width = 640


        
        
        self.patch_embed = nn.Conv2d(in_channels,out_channels=self.embed, kernel_size=self.p_size, stride=self.p_size)
        
        #normalization
        self.norm1 = nn.LayerNorm(self.embed)
        self.norm2 = nn.LayerNorm(self.embed * 2**0)

        #Attention
        self.qkv = nn.Linear(self.dim, self.dim * 3, bias=True)
        self.attn_drop = nn.Dropout(0)

        self.linear = nn.Linear(self.dim, 96)
        self.l_drop = nn.Dropout(0)
        self.softmax = nn.Softmax(dim=-1)

        self.fc1 = nn.Linear(self.dim, 4*self.dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(4*self.dim, self.dim)
        self.drop = nn.Dropout(0)

        # self.change_channels = nn.Conv2d(60, 1, kernel_size=1)

        # Define an upsampling operation that increases the spatial dimensions of the tensor by a factor of 3
        self.upsample = nn.Upsample(size=(self.height, self.width), mode='bilinear', align_corners=True)


        
    def forward(self, x):


        #perform padding for the images
        _, _, H, W = x.size()
        if W % self.patch_size[1] != 0:
            x = F.pad(x, (0, self.patch_size[1] - W % self.patch_size[1]))
        if H % self.patch_size[0] != 0:
            x = F.pad(x, (0, 0, 0, self.patch_size[0] - H % self.patch_size[0]))

        #convolution     
        x = self.patch_embed(x)
        # print("X-patch embed",x.shape)

        #layer norm
        x = x.permute(0,2,3,1)
        x = self.norm1(x)
        x = x.permute(0,3,1,2)
        # print("X-after norm1",x.shape)


        n, c , H , W = x.shape

        #Swin Transformer block
        residual = x
        x = x.permute(0,2,3,1)
        x = self.norm2(x)
        _,H,W,C = x.shape

        # pad images to multiples of window size
        pad_l = pad_t = 0
        pad_r = pad_b = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size
        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b))
        _, Hp, Wp, _ = x.shape
        # print("After padding x ",x.shape)

        new_x = x
        x_windows = window_partition(new_x, self.window_size)
        #change the view to calculate attention
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)
        # print("After window partition",x_windows.shape)


        #Attention mechanism
        B_, N, C = x_windows.shape
        qkv = self.qkv(x)

        #multi-head attention 
        qkv = qkv.reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2] 
        key =  k.transpose(-2, -1)
        #tensor multiplication between q and k
        attention = (q @ key)
        attention = self.softmax(attention)
        attention = self.attn_drop(attention)

        #tensor multiplication between attn and k
        x = (attention @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.linear(x)
        x = self.l_drop(x)
        # print("After attention", x.shape)

        #Merge windows back to picture
        attn_windows = x.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, Hp, Wp)

        x = shifted_x
        residual = residual.permute(0,2,3,1)
        #perform residual padding
        residual = F.pad(residual, (0, 0, pad_l, pad_r, pad_t, pad_b))
        residual = residual.permute(0,3,1,2)
        x = x.permute(0,3,1,2)

        #add residual to x
        x = residual + x

        #cache current x
        residual = x
        x = x.permute(0,2,3,1)
        x = self.norm2(x)

        #perform MLP
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = x.permute(0,3,1,2)
        
        #add residual again
        x = x + residual
        N, C, H, W = x.shape
        x = x.permute(0,2,3,1)

        #Downsample and reverse padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        _,H1,W1,_ = x.shape

        #upsample to get back the image shape 
        x = self.upsample(x)
        # print("After upsample",x.shape)

        #change channels back to depth shape
        # x = self.change_channels(x)
        # print("After upsample",x.shape)
        return x


PSP ( Pyramid Spatial Pooling ) model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PSPModule(nn.Module):
    def __init__(self, in_channels, sizes=(1, 2, 3, 6)):
        super(PSPModule, self).__init__()

        #used to perform 4 different types of adaptive conv by scaling using 1,2,3,6 sizes
        self.stages = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, in_channels // len(sizes), kernel_size=1),
                nn.BatchNorm2d(in_channels // len(sizes)),
                nn.ReLU()
            )
            for size in sizes
        ])
        self.upsample = nn.Conv2d(in_channels=120,out_channels=60,kernel_size = 1)

    def forward(self, x):
        h, w = x.shape[2],x.shape[3]
        features = [x]
        for stage in self.stages:
            out_stage = F.interpolate( #used to upsample and concatenate
                stage(x),
                size=(h, w), 
                mode='bilinear', #uses neighbour four pixels
                align_corners=True
            )
            features.append(out_stage)
        #concatenates the features together
        concat = torch.cat(features, dim=1)
        return(self.upsample(concat))


Decoder SAM model

In [ ]:
class SAM(nn.Module):
    def __init__(self, in_channels,out_channels, window_size=8, shift_size=8):
        super(SAM,self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.window_size = window_size
        self.shift_size = shift_size
        self.num_heads = 8
        self.patch_norm = True
        self.p_size = 4
        self.patch_size = to_2tuple(self.p_size)
        self.embed = 96
        self.dim = self.embed * 2**0
        self.height = 480
        self.width = 640


        
        
        self.patch_embed = nn.Conv2d(in_channels,out_channels=self.embed, kernel_size=self.p_size, stride=self.p_size)
        
        self.norm1 = nn.LayerNorm(self.embed)
        self.norm2 = nn.LayerNorm(self.embed * 2**0)
        #Attention
        self.qkv = nn.Linear(self.dim, self.dim * 3, bias=True)
        self.attn_drop = nn.Dropout(0)
        self.proj = nn.Linear(self.dim, self.dim)
        self.proj_drop = nn.Dropout(0)
        self.softmax = nn.Softmax(dim=-1)

        self.fc1 = nn.Linear(self.dim, 4*self.dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(4*self.dim, self.dim)
        self.drop = nn.Dropout(0)

      
        self.change_channels = nn.Conv2d(60, 1, kernel_size=1)

        # Define an upsampling operation that increases the spatial dimensions of the tensor by a factor of 3
        self.upsample = nn.Upsample(size=(self.height, self.width), mode='bilinear', align_corners=True)


        
    def forward(self, x,y):


        #same as Swin Transformer but twice once for out_swin and other for out_psp
        #-----------------------For E----------------------
        _, _, H, W = x.size()
        if W % self.patch_size[1] != 0:
            x = F.pad(x, (0, self.patch_size[1] - W % self.patch_size[1]))
        if H % self.patch_size[0] != 0:
            x = F.pad(x, (0, 0, 0, self.patch_size[0] - H % self.patch_size[0]))
        x = self.patch_embed(x)


        x = x.permute(0,2,3,1)
        x = self.norm1(x)
        x = x.permute(0,3,1,2)
        # print("X-after norm1",x.shape)


        n, c , H , W = x.shape
        # residual = x
        x = x.permute(0,2,3,1)
        x = self.norm2(x)
        _,H,W,C = x.shape
        pad_l = pad_t = 0
        pad_r = pad_b = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size

        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b))
        _, Hp, Wp, _ = x.shape
        # print("After padding x ",x.shape)

        shifted_x = x
        attn_mask = None
        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)
        # print("After window partition",x_windows.shape)


        #-------------------For Q-----------------------------
        _, _, H, W = y.size()
        if W % self.patch_size[1] != 0:
            y = F.pad(y, (0, self.patch_size[1] - W % self.patch_size[1]))
        if H % self.patch_size[0] != 0:
            y = F.pad(y, (0, 0, 0, self.patch_size[0] - H % self.patch_size[0]))
        y = self.patch_embed(y)
        # print("X-patch embed",x.shape)


        y = y.permute(0,2,3,1)
        y = self.norm1(y)
        y = y.permute(0,3,1,2)
        # print("X-after norm1",x.shape)


        n, c , H , W = y.shape


        #Swin Transformer block
        residual = y
        y = y.permute(0,2,3,1)
        y = self.norm2(y)
        _,H,W,C = y.shape
        pad_l = pad_t = 0
        pad_r = pad_b = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size
        # print(self.window_size)
        # print(pad_r)
        y = F.pad(y, (0, 0, pad_l, pad_r, pad_t, pad_b))
        _, Hp, Wp, _ = y.shape
        # print("After padding x ",x.shape)

        shifted_y = y
        attn_mask = None
        y_windows = window_partition(shifted_y, self.window_size)
        y_windows = y_windows.view(-1, self.window_size * self.window_size, C)

        #-----------CROSS ATTENTION------

        #Attention
        B_,N,C = x_windows.shape #E
        B_1, N1, C1 = y_windows.shape #Q

        qkv_E = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        qkv_Q = self.qkv(y).reshape(B_1, N1, 3, self.num_heads, C1// self.num_heads).permute(2, 0, 3, 1, 4)
        q = qkv_Q[0]
        k ,v = qkv_E[1], qkv_E[2]
        # print("Query",q.shape)
        # print("Key",k.shape)
        # print("value",v.shape)
        attn = (q @ k.transpose(-2, -1))
        attn = self.softmax(attn)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        # print("After attention", x.shape)

        #Merge windows
        attn_windows = x.view(-1, self.window_size, self.window_size, C)
        # print("After merging",attn_windows.shape)
        new_x = window_reverse(attn_windows, self.window_size, Hp, Wp)
        # print("After reverse shift",shifted_x.shape)
        # x = shifted_x

        # x = x.view(n, H * W, C)
        residual = residual.permute(0,2,3,1)
        residual = F.pad(residual, (0, 0, pad_l, pad_r, pad_t, pad_b))
        x = new_x
        residual = residual.permute(0,3,1,2)
        x = x.permute(0,3,1,2)

        #Add residual
        x = residual + x

        residual = x
        x = x.permute(0,2,3,1)
        
        #Perform MLP
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = x.permute(0,3,1,2)
        
        #Add new residual
        x = x + residual

        N, C, H, W = x.shape
        x = x.permute(0,2,3,1)
        #Downsample and reverse padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))

        x0 = x[:, 0::2, 0::2, :] 
        x1 = x[:, 1::2, 0::2, :] 
        x2 = x[:, 0::2, 1::2, :] 
        x3 = x[:, 1::2, 1::2, :]  
        x = torch.cat([x0, x1, x2, x3], -1)  
        _,H1,W1,_ = x.shape
      
        #used to change number of channels
        x = self.change_channels(x)
        # print("After channel switch",x.shape)

        #used to upsample the width and height to original dimensions
        x = self.upsample(x)
        # print("After upsample",x.shape)

        return x

In [ ]:
def compute_errors(gt, pred):

    abs_diff = np.abs(gt - pred)
    # d1_error = abs_diff > (1.25)

    d1 = 0

    rms = (gt - pred) ** 2
    rms = np.sqrt(torch.mean(rms))
    log_rms = (np.log(gt) - np.log(pred)) ** 2
    log_rms = np.sqrt(torch.mean(log_rms))

    abs_rel = torch.mean(np.abs(gt - pred) / gt)
    sq_rel = torch.mean(((gt - pred) ** 2) / gt)
    
    err = np.log(pred) - np.log(gt)
    # silog = np.sqrt(torch.mean(err ** 2) - torch.mean(err) ** 2) * 100
    # print("RMS",err)
    err = np.abs(np.log10(pred) - np.log10(gt))
    log10 = torch.mean(err)
    # print("RMS",log10)

    return [ abs_rel, log10, rms, sq_rel, log_rms, d1]

ResNet block

In [ ]:
import torch.nn as nn
#used layer norm instead of batch norm for batch_size=1
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.LayerNorm(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.LayerNorm(out_channels)
        self.residual_change_conv  = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.residual_change_ln =  nn.LayerNorm(out_channels)

        self.change_channels = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, x):
        residual = x
        x = self.conv1(x)

        x = x.permute(0,2,3,1)

        x = self.bn1(x)
        x = x.permute(0,3,1,2)
        x = self.relu(x)
        x = self.conv2(x)
        x = x.permute(0,2,3,1)

        x = self.bn2(x)
        x = x.permute(0,3,1,2)
        residual = self.residual_change_conv(residual)
        residual = residual.permute(0,2,3,1)

        residual = self.residual_change_ln(residual)
        residual = residual.permute(0,3,1,2)
        x += residual
        x = self.relu(x)

        #change to give input to the swin transformer
        out = self.change_channels(x)
        return out


Training and Validation

In [ ]:
# Training parameters

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

variance_focus = 0.1
learning_rate = 0.01
num_epochs = 5
batch_size = 20

torch.cuda.manual_seed(42)

model = SwinTransformerBlock(in_channels=3,window_size=8).to(device)
model_resnet = ResNetBlock(in_channels=3,out_channels=64).to(device)
model_PSPNet = PSPModule(in_channels=60).to(device) #divisible by all sizes 1,2,3,6
model_SAM = SAM(in_channels=60, out_channels= 1,window_size=8).to(device)


change_channels = nn.Conv2d(60, 1, kernel_size=1)
params_ = list(model.parameters())+ list(model_resnet.parameters())+list(model_PSPNet.parameters()) +list(model_SAM.parameters())
optimizer = torch.optim.Adam([{'params': params_}],lr=learning_rate)
silog_criterion = silog_loss(variance_focus=variance_focus)
huber_loss = HuberLoss(delta=0.5)
train_images = []
train_target = []

for epoch in range(num_epochs):
  # checkpoint_path1 = '/content/drive/MyDrive/PixelTransformer/PixelFormer-main/data_splits/resnet2/checkpoint'+str(epoch)+'.pt'
  # checkpoint_path2 = '/content/drive/MyDrive/PixelTransformer/PixelFormer-main/data_splits/swin2/checkpoint'+str(epoch)+'.pt'
  # checkpoint_path3 = '/content/drive/MyDrive/PixelTransformer/PixelFormer-main/data_splits/psp2/checkpoint'+str(epoch)+'.pt'
  checkpoint_path1 = '/content/drive/MyDrive/CS674-Saha/NewCode/models/resnet2/checkpoint'+str(epoch)+'.pt'
  checkpoint_path2 = '/content/drive/MyDrive/CS674-Saha/NewCode/models/swin2/checkpoint'+str(epoch)+'.pt'
  checkpoint_path3 = '/content/drive/MyDrive/CS674-Saha/NewCode/models/psp2/checkpoint'+str(epoch)+'.pt'
  checkpoint_path4 = '/content/drive/MyDrive/CS674-Saha/NewCode/models/sam2/checkpoint'+str(epoch)+'.pt'
  train_loss = 0
  for val in training_samples:
      image = torch.autograd.Variable(val['image']).to(device) #gpu
      depth_gt = torch.autograd.Variable(val['depth']).to(device)
      # model.train()
      optimizer.zero_grad()
      
      image = image.unsqueeze(0).to(device)
      depth_gt = depth_gt.unsqueeze(0).to(device)

      #image goes through resnet
      resnet_image = model_resnet.train()(image)
      
      #output goes through swin transformer
      out_swin = model.train()(resnet_image) #Swin
      # print("Ater swin",out_swin.shape) #60

      #output from swin goes through PSP 
      out_psp = model_PSPNet.train()(out_swin)
      # depth_est = change_channels(out_psp.cpu()).to(device)
      # print("After psp",out_psp.shape) #120

      #output from PSP and swin goes through Decoder SAM 
      depth_est = model_SAM.train()(out_swin,out_psp)
      # print("After SAM",depth_est.shape) #120

      #used to prevent log 0 error
      mask =  (depth_est>0.1) & (depth_gt>0.1)
      # l1 = silog_criterion.forward(depth_est,depth_gt,mask.to(torch.bool))
      l1 = huber_loss(depth_est[mask],depth_gt[mask])
      # l1 = berhu_loss(depth_est,depth_gt)

      train_loss+=l1
      l1.backward()
      optimizer.step()
      # print("Train loss",l1)
  # model.eval()
  val_loss = 0.0
  val_acc = 0.0
  # Iterate over the validation data loader

  rae ,log10 , rms, sql_rel, log_rms, d1 = 0,0,0,0,0,0

  with torch.no_grad():
      for val in testing_samples:
          image = torch.autograd.Variable(val['image'])
          depth_gt = torch.autograd.Variable(val['depth']) #gpu
          # Forward pass
          image = image.unsqueeze(0).to(device)
          depth_gt = depth_gt.unsqueeze(0).to(device)

          #image goes through resnet
          resnet_image = model_resnet.eval()(image)

          #output goes through swin transformer
          out_swin = model.eval()(resnet_image) #Swin
          # print("Ater swin",out_swin.shape) #60

          #output from swin goes through PSP 
          out_psp = model_PSPNet.eval()(out_swin)
          # depth_est = change_channels(out_psp.cpu()).to(device)
          # print("After psp",out_psp.shape) #120

          #output from PSP and swin goes through Decoder SAM 
          depth_est = model_SAM.eval()(out_swin,out_psp)

          #used to prevent log 0 error
          mask =  (depth_est>0.1) & (depth_gt>0.1)
          # loss = silog_criterion.forward(depth_est, depth_gt,mask.to(torch.bool))
          loss = huber_loss(depth_est[mask],depth_gt[mask])
          # loss = berhu_loss(depth_est,depth_gt)


          val_loss += loss.item() 
          # Compute the accuracy
          erros_measures = compute_errors(depth_est[mask.to(torch.bool)].cpu(),depth_gt[mask.to(torch.bool)].cpu())
          rae += erros_measures[0]
          log10 += erros_measures[1]
          rms += erros_measures[2]
          # sql_rel += erros_measures[3]
          log_rms += erros_measures[4]
          # d1 += erros_measures[5]
      print("Average error measures for val dataset:")
      print("Absolute error {:.4f} Log10 {:.4f} RMS {:.4f}  log_rms {:.4f} ".format(rae/len(testing_samples),log10/len(testing_samples),rms/len(testing_samples),log_rms/len(testing_samples)))
  torch.save(model_resnet.state_dict(), checkpoint_path1)
  torch.save(model.state_dict(), checkpoint_path2)
  torch.save(model_PSPNet.state_dict(), checkpoint_path3)
  torch.save(model_SAM.state_dict(), checkpoint_path4)
  # Compute the average validation loss and accuracy
  val_loss /= len(testing_samples)
  train_loss /=len(training_samples)

  # Print the epoch number, training loss, and validation loss and accuracy
  print("Epoch {}: train_loss {:.4f} val_loss {:.4f}".format(epoch+1, train_loss, val_loss))
    

For Testing purpose

In [27]:
# Training parameters

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.manual_seed(42)
variance_focus = 0.1
learning_rate = 0.01
num_epochs = 5
batch_size = 20

model = SwinTransformerBlock(in_channels=3,window_size=8).to(device)
model_resnet = ResNetBlock(in_channels=3,out_channels=64).to(device)
model_PSPNet = PSPModule(in_channels=60).to(device) #divisible by all sizes 1,2,3,6
model_SAM = SAM(in_channels=60, out_channels= 1,window_size=8).to(device)


change_channels = nn.Conv2d(60, 1, kernel_size=1)

# optimizer = torch.optim.Adam([{'params': model.parameters()}],lr=learning_rate)
silog_criterion = silog_loss(variance_focus=variance_focus)
huber_loss = HuberLoss(delta=0.5)

#for best checkpoint

# for epoch in range(num_epochs):
train_loss = 0
model_resnet.load_state_dict(torch.load('checkpoint-resnet.pt'))
model.load_state_dict(torch.load('checkpoint-swin.pt'))
model_PSPNet.load_state_dict(torch.load('checkpoint-psp.pt'))
model_SAM.load_state_dict(torch.load('checkpoint-sam.pt'))

for param in model.parameters():
    param.requires_grad = False

for param in model_resnet.parameters():
    param.requires_grad = False

for param in model_PSPNet.parameters():
    param.requires_grad = False


for param in model_SAM.parameters():
    param.requires_grad = False


val_loss = 0.0
val_acc = 0.0
# Iterate over the validation data loader

rae ,log10 , rms, sql_rel, log_rms, d1 = 0,0,0,0,0,0

with torch.no_grad():
    for val in testing_samples:
        image = torch.autograd.Variable(val['image'])
        depth_gt = torch.autograd.Variable(val['depth']) #gpu
        # Forward pass
        image = image.unsqueeze(0).to(device)
        depth_gt = depth_gt.unsqueeze(0).to(device)

        #image goes through resnet
        resnet_image = model_resnet.eval()(image)

        #output goes through swin transformer
        out_swin = model.eval()(resnet_image) #Swin
        # print("Ater swin",out_swin.shape) #60

        #output from swin goes through PSP 
        out_psp = model_PSPNet.eval()(out_swin)
        # depth_est = change_channels(out_psp.cpu()).to(device)
        # print("After psp",out_psp.shape) #120

        #output from PSP and swin goes through Decoder SAM 
        depth_est = model_SAM.eval()(out_swin,out_psp)

        #used to prevent log 0 error
        mask =  (depth_est>0.1) & (depth_gt>0.1)
        # loss = silog_criterion.forward(depth_est, depth_gt,mask.to(torch.bool))
        loss = huber_loss(depth_est[mask],depth_gt[mask])
        # loss = berhu_loss(depth_est,depth_gt)


        val_loss += loss.item() 
        #print(val_loss)
        # Compute the accuracy
        erros_measures = compute_errors(depth_est[mask.to(torch.bool)].cpu(),depth_gt[mask.to(torch.bool)].cpu())
        rae += erros_measures[0]
        log10 += erros_measures[1]
        rms += erros_measures[2]
        # sql_rel += erros_measures[3]
        log_rms += erros_measures[4]
        # d1 += erros_measures[5]
    print("Average error measures for val dataset:")
    print("Absolute error {:.4f} Log10 {:.4f} RMS {:.4f}  log_rms {:.4f} ".format(rae/len(testing_samples),log10/len(testing_samples),rms/len(testing_samples),log_rms/len(testing_samples)))
# Compute the average validation loss and accuracy
val_loss /= len(testing_samples)
# train_loss /=len(training_samples)

# Print the epoch number, training loss, and validation loss and accuracy
print(" val_loss {:.4f}".format( val_loss))
  

Average error measures for val dataset:
Absolute error 0.3797 Log10 0.1538 RMS 1.1649  log_rms 0.4244 
 val_loss 0.3552
